## LSTM

In [1]:
import os
import time
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset
from zipfile import ZipFile

In [2]:
class SkeletonDataSet:
    
    files = None
    action_classes = None
    
    def __init__(self, data_path, broken_files_path , chonk_len):
        self.data_path = data_path
        self.broken_files_path = broken_files_path
        self.chonk_len = chonk_len
        self.training_subjects = list(range(0, 28))
        self.training_cameras = [1, 2, 3]
        self.training_classes = [5, 7, 20, 24, 27, 33, 38, 50, 72, 93]
    
    def read_data(self):
        labels = []
        files = []
        action_classes = {}
        counter = 0
        files_counter = {}

        with open(self.broken_files_path, 'r') as f:
            broken_files = f.read().split("\n")

        raw_files = os.listdir(self.data_path)
        num_frames = 0

        for filename in raw_files:
            if filename not in broken_files:
                action_class = int(filename[filename.find('A') + 1:filename.find('A') + 4])
                subject_id = int(filename[filename.find('P') + 1:filename.find('P') + 4])
                camera_id = int(filename[filename.find('C') + 1:filename.find('C') + 4])
                if action_class in self.training_classes and camera_id in self.training_cameras:  # and subject_id in training_subjects:
                    if action_class in action_classes:
                        if files_counter[action_class] < 120:
                            files.append([filename, action_classes[action_class]])
                            files_counter[action_class] = files_counter[action_class] + 1
                    else:
                        action_classes.update({action_class : counter})
                        files_counter.update({action_class : 1})
                        counter+=1
                        files.append([filename,action_classes[action_class]])
                        labels.append([action_class])
                        
        print("action classes: ", action_classes)
        print("action files: ", files_counter)
        
        self.files = files
        self.action_classes = action_classes
        
        return files, action_classes
    
    def get_nonzero_std(self, s): 
        index = s.sum(-1).sum(-1) != 0  
        s = s[index]
        if len(s) != 0:
            s = s[:, :, 0].std() + s[:, :, 1].std() + s[:, :, 2].std()  
        else:
            s = 0
        return s
    
    def read_skeleton_filter(self, file):
        with open(file, 'r') as f:
            skeleton_sequence = {}
            skeleton_sequence['numFrame'] = int(f.readline())
            skeleton_sequence['frameInfo'] = []
            for t in range(skeleton_sequence['numFrame']):
                frame_info = {}
                frame_info['numBody'] = int(f.readline())
                frame_info['bodyInfo'] = []

                for m in range(frame_info['numBody']):
                    body_info = {}
                    body_info_key = [
                        'bodyID', 'clipedEdges', 'handLeftConfidence',
                        'handLeftState', 'handRightConfidence', 'handRightState',
                        'isResticted', 'leanX', 'leanY', 'trackingState'
                    ]
                    body_info = {
                        k: float(v)
                        for k, v in zip(body_info_key, f.readline().split())
                    }
                    body_info['numJoint'] = int(f.readline())
                    body_info['jointInfo'] = []
                    for v in range(body_info['numJoint']):
                        joint_info_key = [
                            'x', 'y', 'z', 'depthX', 'depthY', 'colorX', 'colorY',
                            'orientationW', 'orientationX', 'orientationY',
                            'orientationZ', 'trackingState'
                        ]
                        joint_info = {
                            k: float(v)
                            for k, v in zip(joint_info_key, f.readline().split())
                        }
                        body_info['jointInfo'].append(joint_info)
                    frame_info['bodyInfo'].append(body_info)
                skeleton_sequence['frameInfo'].append(frame_info)

        return skeleton_sequence
    
    def read_xyz(self, file, max_body=1, num_joint=25):
        seq_info = self.read_skeleton_filter(file)
        data = np.zeros((max_body, seq_info['numFrame'], num_joint, 3))
        for n, f in enumerate(seq_info['frameInfo']):
            for m, b in enumerate(f['bodyInfo']):
                for j, v in enumerate(b['jointInfo']):
                    if m < max_body and j < num_joint:
                        data[m, n, j, :] = [v['x'], v['y'], v['z']]

                    else:
                        pass

        return data
    
    def create_coords_blocks(self, test_file):   
        frame_counter = 0
        new_labels = []
        new_frames = []
        blocks = []

        test_frames = self.read_xyz(self.data_path + test_file[0])[0]
        label = test_file[1]
        slice_len = self.chonk_len * int(len(test_frames) / self.chonk_len)


        for index in range(len(test_frames[:slice_len])):
            frame_counter += 1
            new_frames.append(test_frames[index].flatten())
            if frame_counter == self.chonk_len:
                frame_counter = 0
                blocks.append(np.array(new_frames))
                new_labels = new_labels + [label]
                new_frames = []


        return blocks, new_labels
    
    def mark_data(self):
        data = []
        labels = []
        ##########################################################################
        numbers = {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0} #####
        ##################################################################
        for file in self.files:
            frames_blocks, label = self.create_coords_blocks(file)
            if label != [] and numbers[label[0]] <= 150:
                numbers[label[0]] += len(label)
                data += frames_blocks
                labels += label
        data_np = np.asarray(data)
        labels_np = np.asarray(labels)

        data_sq = data_np.reshape(len(data_np), -1)
        test_data = pd.DataFrame(data_sq)
        test_labels = pd.DataFrame(labels_np)
        test_data['labels'] = test_labels
        
        return test_data

In [3]:
DATA_PATH = 'C:\\Users\\butoc\\Study\\nturgb+d_skeletons\\' 
BROKEN_FILES_PATH = r'C:\Users\butoc\Study\NTU_RGBD_samples_with_missing_skeletons.txt'

In [4]:
chonk_len = 50
data_skeleton = SkeletonDataSet(data_path=DATA_PATH, broken_files_path=BROKEN_FILES_PATH, chonk_len=chonk_len)

data_skeleton.read_data()

df = data_skeleton.mark_data()

action classes:  {5: 0, 7: 1, 20: 2, 24: 3, 27: 4, 33: 5, 38: 6, 50: 7}
action files:  {5: 120, 7: 120, 20: 120, 24: 120, 27: 120, 33: 120, 38: 120, 50: 120}


In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,3741,3742,3743,3744,3745,3746,3747,3748,3749,labels
0,0.216129,0.185681,3.767784,0.228515,0.437528,3.691366,0.239707,0.684526,3.604562,0.209585,...,0.003934,0.357284,3.581750,0.157428,0.392301,3.460259,0.171550,0.370148,3.459167,0
1,0.237631,0.235039,3.746806,0.246957,0.468998,3.670609,0.255474,0.698446,3.586321,0.227848,...,-0.285979,0.445472,3.509000,-0.022870,0.701316,2.982738,-0.008102,0.664465,2.972972,1
2,0.443868,-0.031092,4.133648,0.434343,0.196818,4.022484,0.423121,0.421168,3.902824,0.405972,...,0.172790,0.540568,3.877889,0.538015,0.285437,3.607140,0.541454,0.294485,3.597500,2
3,0.441684,0.011612,4.112082,0.428414,0.239035,4.017493,0.415520,0.462060,3.915770,0.396433,...,0.199185,-0.086971,3.853000,0.419361,-0.082200,3.837892,0.410779,-0.026258,3.839000,2
4,0.286811,0.168139,3.734849,0.288774,0.415689,3.650531,0.290175,0.658193,3.557110,0.260253,...,-0.007122,0.151750,3.826333,0.474957,0.068992,3.611121,0.488455,0.128277,3.603967,3


In [6]:
df.to_csv('skeleton50.csv', index = False)

In [7]:
chonk_len = 25
data_skeleton = SkeletonDataSet(data_path=DATA_PATH, broken_files_path=BROKEN_FILES_PATH, chonk_len=chonk_len)

data_skeleton.read_data()

df = data_skeleton.mark_data()

action classes:  {5: 0, 7: 1, 20: 2, 24: 3, 27: 4, 33: 5, 38: 6, 50: 7}
action files:  {5: 120, 7: 120, 20: 120, 24: 120, 27: 120, 33: 120, 38: 120, 50: 120}


In [8]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1866,1867,1868,1869,1870,1871,1872,1873,1874,labels
0,0.216129,0.185681,3.767784,0.228515,0.437528,3.691366,0.239707,0.684526,3.604562,0.209585,...,0.009154,0.378643,3.572429,0.189420,0.300753,3.439293,0.247817,0.307582,3.417111,0
1,0.220766,0.173490,3.765520,0.224977,0.422768,3.688795,0.228065,0.665202,3.601240,0.188255,...,0.003934,0.357284,3.581750,0.157428,0.392301,3.460259,0.171550,0.370148,3.459167,0
2,0.237631,0.235039,3.746806,0.246957,0.468998,3.670609,0.255474,0.698446,3.586321,0.227848,...,0.032369,0.166263,3.831250,0.360257,0.212299,3.393962,0.380253,0.250230,3.393125,1
3,0.239552,0.230314,3.749655,0.245591,0.467265,3.672571,0.250260,0.699394,3.585419,0.225500,...,-0.285979,0.445472,3.509000,-0.022870,0.701316,2.982738,-0.008102,0.664465,2.972972,1
4,0.015030,0.193286,3.639623,-0.018082,0.457431,3.553125,-0.054591,0.717096,3.455672,-0.043707,...,-0.440409,0.240558,3.748333,0.005636,0.116553,3.157196,0.016155,0.151978,3.171000,1


In [9]:
df.to_csv('skeleton25.csv', index = False)

In [10]:
chonk_len = 75
data_skeleton = SkeletonDataSet(data_path=DATA_PATH, broken_files_path=BROKEN_FILES_PATH, chonk_len=chonk_len)

data_skeleton.read_data()

df = data_skeleton.mark_data()

action classes:  {5: 0, 7: 1, 20: 2, 24: 3, 27: 4, 33: 5, 38: 6, 50: 7}
action files:  {5: 120, 7: 120, 20: 120, 24: 120, 27: 120, 33: 120, 38: 120, 50: 120}


In [11]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,5616,5617,5618,5619,5620,5621,5622,5623,5624,labels
0,0.237631,0.235039,3.746806,0.246957,0.468998,3.670609,0.255474,0.698446,3.586321,0.227848,...,-0.440409,0.240558,3.748333,0.005636,0.116553,3.157196,0.016155,0.151978,3.171000,1
1,0.443868,-0.031092,4.133648,0.434343,0.196818,4.022484,0.423121,0.421168,3.902824,0.405972,...,0.256465,0.605232,3.891000,0.609450,0.359791,3.615084,0.603515,0.374389,3.606429,2
2,0.286811,0.168139,3.734849,0.288774,0.415689,3.650531,0.290175,0.658193,3.557110,0.260253,...,-0.010432,0.125070,3.759000,0.451703,0.057018,3.610559,0.428588,0.128333,3.614375,3
3,0.284738,0.178680,3.764987,0.299845,0.428857,3.687624,0.314031,0.673921,3.600093,0.296217,...,-0.175307,0.287524,3.488758,0.169603,0.762664,3.404888,0.160853,0.747544,3.400697,4
4,0.235327,0.154995,3.763243,0.248227,0.409883,3.677307,0.260952,0.660343,3.581986,0.242137,...,0.152575,0.530291,3.324611,0.143551,0.530439,3.339436,0.193809,0.472986,3.423596,5


In [12]:
df.to_csv('skeleton75.csv', index = False)